# 📊 Validação de Modelos de Séries Temporais - CVC Lojas

## 🎯 Objetivo Executivo
Este notebook tem como objetivo realizar a **validação robusta (Backtesting)** de múltiplos algoritmos de previsão de vendas para as lojas da CVC. O processo simula cenários reais do passado para garantir que o modelo escolhido tenha performance consistente ao longo do tempo, e não apenas em um único período de teste.

## 🛠️ Metodologia: Walk-Forward Validation (Strict Mode)
Diferente da divisão tradicional (Treino/Teste), utilizamos a estratégia de **Walk-Forward** (Janela Deslizante):
1.  O modelo treina com dados até uma data de corte (ex: Dez/2024).
2.  Faz a previsão para o mês seguinte (ex: Jan/2025).
3.  A janela avança 1 mês, o modelo retreina com os dados reais de Jan/2025 e prevê Fev/2025.
4.  Isso se repete por 12 meses (Folds), gerando métricas de erro (RMSE, SMAPE) para cada mês.

> **Nota:** O modo "Strict" garante que **nenhum dado do futuro** (vazamento de dados) seja acessível ao modelo durante o treino, simulando fielmente a produção.

---

## 🤖 Estratégia de Modelos (Model)
A pipeline avalia automaticamente duas classes de algoritmos via biblioteca **Darts**:

### 1. Machine Learning Clássico (Regressores)
* **Linear Regression:** Baseline simples para capturar tendências lineares.
* **Random Forest:** Captura não-linearidades e interações complexas.
* **LightGBM / XGBoost / CatBoost:** Modelos baseados em *Gradient Boosting*, estado da arte para dados tabulares e séries temporais com covariáveis.

### 2. Deep Learning (SOTA - State of the Art)
* **TFT (Temporal Fusion Transformer):** Modelo de atenção que aprende a importância de cada variável ao longo do tempo.
* **N-BEATS:** Rede neural baseada em blocos de tendência e sazonalidade.
* **Transformer:** Arquitetura clássica de *Attention* adaptada para séries temporais.
* **BlockRNN (LSTM):** Redes recorrentes para capturar dependências de longo prazo.
* **TCN (Temporal Convolutional Network):** Convoluções causais para capturar padrões locais e globais.

---

## 🏛️ Arquitetura e Governança (Databricks Unity Catalog)
Este notebook implementa uma arquitetura híbrida para conformidade com o Unity Catalog:

| Componente | Local de Armazenamento | Função |
| :--- | :--- | :--- |
| **Experimentos** | `Workspace/Users/...` | Armazena métricas, gráficos e logs de execução (evita erro de path do UC). |
| **Registro de Modelos** | **Unity Catalog** (`ds_dev.cvc_val`) | O modelo final (`.pkl`) é versionado e governado oficialmente no catálogo. |
| **Assinatura (Signature)** | **Enforced** | Todos os modelos possuem contrato de entrada/saída (`long` -> `double`) validado para evitar erros de tipagem no serving. |

## 📥 Dados de Entrada
* **Target:** `bip_vhistorico_targuet_loja` (Vendas históricas).
* **Covariáveis Futuras:** `bip_vhistorico_feriados_loja` (Calendário nacional/regional).
* **Covariáveis Globais:** `bip_vhistorico_suporte_canal_loja` (Indicadores macroeconômicos e campanhas).

In [0]:
import os
import pickle
import warnings
import traceback
from datetime import datetime
import pytz
import pandas as pd
import numpy as np
import mlflow
from mlflow.models import ModelSignature
from mlflow.types.schema import Schema, ColSpec
import mlflow.sklearn
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import hashlib
import json

# Darts Imports
from darts import TimeSeries
from darts.dataprocessing.pipeline import Pipeline
from darts.dataprocessing.transformers import (
    Scaler,
    StaticCovariatesTransformer,
    MissingValuesFiller
)
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.models import (
    TFTModel,
    NBEATSModel,
    TransformerModel,
    LinearRegressionModel,
    LightGBMModel,
    XGBModel,
    CatBoostModel,
    RandomForest,
    BlockRNNModel,
    RNNModel,
    TCNModel
)
from darts.metrics import mape, mse, rmse, r2_score, smape
from pytorch_lightning.callbacks import EarlyStopping

# Configuração do Ambiente
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

# Obtém Spark Session
try:
    spark = SparkSession.builder.getOrCreate()
except NameError:
    spark = None # Lidar com isso se rodar fora do spark
warnings.filterwarnings("ignore")

In [0]:
# 1. CONFIGURAÇÃO (SINGLETON) ---
class Config:
    def __init__(self):
        # Tenta pegar widgets do Databricks
        try:
            from pyspark.dbutils import DBUtils
            dbutils = DBUtils(spark)
        except ImportError:
            dbutils = None

        try:
            # Setup Widgets se não existirem
            try:
                dbutils.widgets.text("data_inicio_treino", "2019-01-01", "1. Início Treino")
                dbutils.widgets.text("data_fim_treino", "2025-01-01", "2. Fim Treino (Corte)")
                dbutils.widgets.text("data_fim_validacao", "2025-12-31", "3. Fim Validação (Ground Truth)")
                dbutils.widgets.text("catalog", "ds_dev", "4. Catálogo")
                dbutils.widgets.text("forecast_horizon", "35", "5. Horizonte (Dias)")
                dbutils.widgets.text("n_epochs", "20", "6. Épocas (DL Models)")
                dbutils.widgets.text("lags", "5", "7. Lags")
            except:
                pass # Widgets já criados ou erro ao criar

            self.CATALOG = dbutils.widgets.get("catalog")
            self.DATA_START = dbutils.widgets.get("data_inicio_treino")
            # TRAIN_END_DATE: Data limite para o modelo APRENDER (Fit).
            self.TRAIN_END_DATE = dbutils.widgets.get("data_fim_treino")
            # INGESTION_END: Data limite para carregar dados (precisa incluir 2025 para validar)
            self.INGESTION_END = dbutils.widgets.get("data_fim_validacao")
            self.VAL_START_DATE = self.TRAIN_END_DATE                               # Validação começa onde treino termina
            self.FORECAST_HORIZON = int(dbutils.widgets.get("forecast_horizon"))
            self.N_EPOCHS = int(dbutils.widgets.get("n_epochs"))
            self.LAGS = int(dbutils.widgets.get("lags"))
        except (NameError, Exception):
            print('⚠️ Célula rodando fora do contexto de Widgets do Databricks/dbutils.')

        self.SCHEMA = "cvc_val"
        # Unity Catalog Volumes Path
        self.VOLUME_ROOT = f"/Volumes/{self.CATALOG}/{self.SCHEMA}/experiments/artefacts/loja"
        self.PATH_SCALERS = f"{self.VOLUME_ROOT}/scalar/validation"
        self.PATH_MODELS = f"{self.VOLUME_ROOT}/models/validation"
        
        # DEFINIÇÃO DO EXPERIMENTO MLFLOW
        self.EXPERIMENT_NAME = "/Workspace/Shared/data_science/projetos/cvc_curva_de_vendas_por_canal/experiments/Model_Validation_CVC_Loja"
        self.MLFLOW_REGISTRY_URI = "databricks-uc"
        self.LAGS_FUTURE = [0, -1, -2, -3]
        self.VERSION = datetime.now(pytz.timezone('America/Sao_Paulo')).strftime('%Y_%m_%d_%H_%M')

        # Garante estrutura de diretórios
        for path in [self.PATH_SCALERS, self.PATH_MODELS]:
            os.makedirs(path, exist_ok=True)

In [0]:
# --- 2. INGESTÃO DE DADOS (ETL NO SPARK - SCALABLE) ---
from databricks.feature_engineering import FeatureEngineeringClient, FeatureLookup
import pyspark.sql.functions as F
import pandas as pd
import numpy as np
from darts import TimeSeries
from darts.utils.timeseries_generation import datetime_attribute_timeseries

class DataIngestion:
    def __init__(self, spark_session, config):
        self.spark = spark_session
        self.config = config
        self.fe = FeatureEngineeringClient()

    def create_training_set(self):
        """
        Gera o dataset completo via Feature Store e realiza ETL nativo no Spark.
        Retorna: DataFrame PySpark (Lazy Evaluation)
        """
        print("🛒 Construindo Training Set via Feature Store (Spark Native)...")

        # 1. Definir a 'Spine' (Target)
        target_table = f"{self.config.CATALOG}.{self.config.SCHEMA}.bip_vhistorico_targuet_loja"
        
        df_spine = (self.spark.table(target_table)
                    .filter(F.col("DATA").between(self.config.DATA_START, self.config.INGESTION_END))
                    .select("CODIGO_LOJA", "DATA", "VALOR")
                    .withColumnRenamed("VALOR", "TARGET_VENDAS")
                    .withColumn("CODIGO_LOJA", F.col("CODIGO_LOJA").cast("string"))
                   )

        # 2. Configurar Lookups
        feature_lookups = [
            FeatureLookup(
                table_name=f"{self.config.CATALOG}.{self.config.SCHEMA}.cmc_alojas",
                lookup_key=["CODIGO_LOJA"],
                feature_names=["CLUSTER_LOJA", "SIGLA_UF", "TIPO_LOJA", "MODELO_LOJA"]
            ),
            FeatureLookup(
                table_name=f"{self.config.CATALOG}.{self.config.SCHEMA}.bip_vhistorico_feriados_loja",
                lookup_key=["CODIGO_LOJA"],
                timestamp_lookup_key="DATA",
                feature_names=["VALOR"], 
                output_name="IS_FERIADO"
            )
        ]

        # 3. Criar Training Set (Retorna objeto FeatureEngineeringTrainingSet)
        training_set = self.fe.create_training_set(
            df=df_spine,
            feature_lookups=feature_lookups,
            label="TARGET_VENDAS",
            exclude_columns=[]
        )

        # 4. Carregar como DataFrame Spark (SEM toPandas aqui)
        df_spark = training_set.load_df()

        # --- ETL NATIVO NO SPARK (Distribuído) ---
        print("   ⚡ Executando limpeza e tratamento no Spark Cluster...")
        
        # A. Tratamento de Nulos (Left Joins geram nulos)
        # Sintaxe Spark: fill(valor, subset=[colunas]) ou fill({col: val})
        df_spark = df_spark.na.fill({
            "IS_FERIADO": 0.0, 
            "TARGET_VENDAS": 0.0,
            "CLUSTER_LOJA": "DESCONHECIDO",
            "SIGLA_UF": "DESCONHECIDO",
            "TIPO_LOJA": "DESCONHECIDO",
            "MODELO_LOJA": "DESCONHECIDO"
        })

        # B. Garantia de Tipos (Casting)
        df_spark = df_spark.withColumn("DATA", F.to_timestamp("DATA"))
        
        return df_spark

    def get_global_support(self):
        """
        Carrega suporte global mantendo processamento no Spark até o final.
        """
        table_name = "bip_vhistorico_suporte_canal_loja"
        print(f"🌍 Carregando suporte global (Spark Aggregation)...")
        
        # Toda a agregação ocorre no cluster
        df_spark = (self.spark.table(f"{self.config.CATALOG}.{self.config.SCHEMA}.{table_name}")
            .filter(F.col("DATA").between(self.config.DATA_START, self.config.INGESTION_END))
            .groupBy("DATA")
            .pivot("METRICAS")
            .agg(F.sum("VALOR"))
            .na.fill(0.0) # Preenche nulos do pivot no Spark
        )
        
        # Só converte o resultado final (pequeno) para Pandas
        pdf = df_spark.toPandas()
        pdf['DATA'] = pd.to_datetime(pdf['DATA'])
        return pdf.set_index('DATA').asfreq('D').fillna(0.0)

    def build_darts_objects(self, df_spark_wide, df_global_support):
        """
        Recebe Spark DataFrame -> Converte para Pandas -> Cria objetos Darts
        """
        print("⚙️ Materializando dados do Spark para Pandas (Driver)...")
        
        # AQUI acontece a transferência de dados Cluster -> Driver
        # Como já filtramos e limpamos no Spark, o dado vem menor e mais limpo.
        df_wide = df_spark_wide.toPandas()
        
        # Garante tipos Pandas compatíveis com Darts
        df_wide['DATA'] = pd.to_datetime(df_wide['DATA'])
        
        # Identificação dinâmica de colunas estáticas
        possible_static = ["CLUSTER_LOJA", "SIGLA_UF", "TIPO_LOJA", "MODELO_LOJA"]
        static_cols = [c for c in possible_static if c in df_wide.columns]

        print("   Build: Criando Target Series...")
        target_series_list = TimeSeries.from_group_dataframe(
            df_wide,
            group_cols="CODIGO_LOJA",
            time_col="DATA",
            value_cols="TARGET_VENDAS",
            static_cols=static_cols,
            freq='D',
            fill_missing_dates=True,
            fillna_value=0.0
        )
        
        target_dict = {str(ts.static_covariates.index[0]): ts for ts in target_series_list}
        valid_stores = list(target_dict.keys())

        print("   Build: Criando Covariáveis Locais...")
        feriado_series_list = TimeSeries.from_group_dataframe(
            df_wide,
            group_cols="CODIGO_LOJA",
            time_col="DATA",
            value_cols=["IS_FERIADO"], 
            freq='D',
            fill_missing_dates=True,
            fillna_value=0.0
        )
        feriado_dict = {str(ts.static_covariates["CODIGO_LOJA"].iloc[0]): ts for ts in feriado_series_list}

        # Globais (já vieram prontas do método get_global_support)
        ts_support = TimeSeries.from_dataframe(
            df_global_support, 
            fill_missing_dates=True, 
            freq='D',
            fillna_value=0.0
        )
        
        ts_time = datetime_attribute_timeseries(df_global_support.index, attribute="dayofweek", cyclic=True)
        ts_time = ts_time.stack(datetime_attribute_timeseries(df_global_support.index, attribute="quarter", one_hot=True))
        ts_time = ts_time.stack(datetime_attribute_timeseries(df_global_support.index, attribute="week", cyclic=True))
        
        global_covariates = ts_support.stack(ts_time)

        final_target_list = []
        full_covariates_list = []

        print("   Build: Stacking Final...")
        for loja in valid_stores:
            ts_target = target_dict[loja]
            final_target_list.append(ts_target)
            
            ts_local = feriado_dict.get(loja)
            if ts_local:
                 ts_local = ts_local.slice_intersect(ts_target)
            else:
                 ts_local = TimeSeries.from_times_and_values(ts_target.time_index, np.zeros(len(ts_target)), freq='D')
            
            ts_global = global_covariates.slice_intersect(ts_target)
            full_covariates_list.append(ts_global.stack(ts_local))

        print(f"✅ Objetos Darts Prontos: {len(final_target_list)} lojas.")
        return final_target_list, full_covariates_list

In [0]:
# --- 3. FEATURE ENGINEERING (UNIFIED PIPELINE) ---
class ProjectPipeline:
    def __init__(self):
        self.target_pipeline = Pipeline([
            MissingValuesFiller(verbose=False),
            Scaler(name="target_scaler")
        ])
        self.static_pipeline = Pipeline([
            StaticCovariatesTransformer(verbose=False)
        ])
        self.covariate_pipeline = Pipeline([
            MissingValuesFiller(verbose=False),
            Scaler(name="covar_scaler")
        ])

    def fit(self, target_series, covariates):
        self.target_pipeline.fit(target_series)
        self.static_pipeline.fit(target_series) 
        self.covariate_pipeline.fit(covariates)
        return self

    def transform(self, target_series, covariates):
        ts_scaled = self.target_pipeline.transform(target_series)
        ts_scaled = self.static_pipeline.transform(ts_scaled)
        cov_scaled = self.covariate_pipeline.transform(covariates)
        return ts_scaled, cov_scaled

    def inverse_transform(self, target_series, partial=False):
        return self.target_pipeline.inverse_transform(target_series, partial=partial)

In [0]:
# --- 4. MODEL TRAINING MANAGER (Walk-Forward V2 - Enriched Logging) ---
class DartsWrapper(mlflow.pyfunc.PythonModel):
    """
    Wrapper melhorado para suportar inferencia com covariaveis globais.
    """
    def load_context(self, context):
        import pickle
        with open(context.artifacts["darts_model"], "rb") as f:
            self.model = pickle.load(f)
        
        self.future_covariates = None
        if "future_covariates" in context.artifacts:
            try:
                with open(context.artifacts["future_covariates"], "rb") as f:
                    self.future_covariates = pickle.load(f)
            except Exception as e:
                print(f"⚠️ [Wrapper] Erro ao carregar covariáveis: {e}")

    def predict(self, context, model_input):
        import pandas as pd
        import numpy as np
        
        n = 1
        if isinstance(model_input, pd.DataFrame):
            if 'n' in model_input.columns:
                try:
                    n = int(model_input.iloc[0]['n'])
                except:
                    pass
        
        predict_kwargs = {"n": n}
        if self.model.supports_future_covariates and self.future_covariates is not None:
             predict_kwargs["future_covariates"] = self.future_covariates

        try:
            pred = self.model.predict(**predict_kwargs)
            if isinstance(pred, list):
                 return pd.concat([p.pd_dataframe() for p in pred])
            return pred.pd_dataframe()
        except Exception as e:
            print(f"⚠️ [Wrapper] Falha na predição: {str(e)}. Retornando Dummy.")
            dates = pd.date_range(start="2025-01-01", periods=n, freq="D")
            return pd.DataFrame(np.zeros((n, 1)), index=dates, columns=["dummy_prediction"])

class ModelTrainer:
    def __init__(self, config, models_dict):
        self.config = config
        self.models = models_dict
        self.success_models = []
        self.failed_models = []

    def _get_store_ids(self, series_list):
        """Extrai IDs das lojas das TimeSeries de forma segura"""
        ids = []
        for ts in series_list:
            try:
                # Tenta pegar do índice das covariáveis estáticas (padrão Darts Group)
                if ts.static_covariates is not None:
                    ids.append(str(ts.static_covariates.index[0]))
                else:
                    ids.append("UNKNOWN")
            except:
                ids.append("ERROR")
        return ids

    def train_evaluate_walkforward(self, train_series_static, train_covs_static, full_series_scaled, full_covariates_scaled, val_series_original, target_pipeline):
        """
        Executa treinamento estático (2024) e validação mensal progressiva (2025).
        """
        mlflow.set_experiment(self.config.EXPERIMENT_NAME)
        
        # Prepara metadados das lojas
        store_ids = self._get_store_ids(train_series_static)
        store_ids_str = ",".join(store_ids)
        # Cria um hash curto para identificar o conjunto de lojas nos parâmetros
        stores_hash = hashlib.md5(store_ids_str.encode()).hexdigest()[:8]

        # Salva covariáveis completas para artefato (usado no wrapper)
        covariates_path = f"{self.config.VOLUME_ROOT}/temp/temp_future_covariates_v{self.config.VERSION}.pkl"
        with open(covariates_path, "wb") as f:
            pickle.dump(full_covariates_scaled, f)
        
        # Datas de corte para Walk-Forward
        validation_range = pd.date_range(start=self.config.VAL_START_DATE, end=self.config.INGESTION_END, freq='MS')

        for model_name, model in self.models.items():
            print(f"\n🚀 [Model: {model_name}] Iniciando Processo...")
            model_metrics_global = {}
            all_predictions = []
            
            try:
                with mlflow.start_run(run_name=f"{model_name}_v{self.config.VERSION}") as run:
                    # --- PARTE 1: LOGGING DE METADADOS RICOS ---
                    print(f"   📝 Registrando metadados do experimento...")
                    
                    # 1. Parâmetros de Configuração Básica
                    mlflow.log_param("version", self.config.VERSION)
                    mlflow.log_param("horizon", self.config.FORECAST_HORIZON)
                    mlflow.log_param("model_name", model_name)
                    mlflow.log_param("lags", self.config.LAGS)
                    mlflow.log_param("epochs", self.config.N_EPOCHS)
                    
                    # 2. Intervalos de Datas (Fundamental para reprodutibilidade)
                    mlflow.log_param("data_start_date", self.config.DATA_START)
                    mlflow.log_param("train_cutoff_date", self.config.TRAIN_END_DATE)
                    mlflow.log_param("validation_start", self.config.VAL_START_DATE)
                    mlflow.log_param("validation_end", self.config.INGESTION_END)
                    
                    # 3. Contexto das Lojas (Dataset Info)
                    mlflow.log_param("n_stores_trained", len(store_ids))
                    mlflow.log_param("stores_hash", stores_hash)
                    
                    # Salva a lista completa de lojas como arquivo de texto (Artifact)
                    mlflow.log_text(store_ids_str, "metadata/trained_store_ids.txt")
                    
                    # 4. Hiperparâmetros do Modelo Específico
                    if hasattr(model, 'model_params'):
                         mlflow.log_param("model_internal_params", str(model.model_params)[:250])

                    # --- PARTE 2: TREINAMENTO ESTÁTICO ---
                    print(f"   🏋️ Treinando com dados até {self.config.TRAIN_END_DATE}...")
                    
                    kwargs = {}
                    if model.supports_past_covariates:
                        kwargs['past_covariates'] = train_covs_static
                    if model.supports_future_covariates:
                        kwargs['future_covariates'] = train_covs_static
                    
                    model.fit(train_series_static, **kwargs)
                    
                    # Salvar Modelo e Registrar no MLflow (Estado Base)
                    filename = f"{model_name}_v{self.config.VERSION}.pkl"
                    local_path = f"{self.config.PATH_MODELS}/{filename}"
                    model.save(local_path)                  

                    # ========================================================
                    # AQUI ESTÁ A ALTERAÇÃO PARA REGISTRAR NO UNITY CATALOG
                    # ========================================================
                    full_model_name = f"{self.config.CATALOG}.{self.config.SCHEMA}.loja_{model_name}"
                    
                    artifacts = {"darts_model": local_path}
                    if model.supports_future_covariates:
                        artifacts["future_covariates"] = covariates_path                    

                    input_schema = Schema([ColSpec("long", "n")]) 
                    output_schema = Schema([ColSpec("double", "prediction")])
                    signature = ModelSignature(inputs=input_schema, outputs=output_schema)
                    
                    # Tags para facilitar busca visual no MLflow UI
                    mlflow.set_tag("model_type", model_name)
                    mlflow.set_tag("validation_mode", "walk-forward-strict")

                    print(f"   💾 Registrando modelo como: {full_model_name}")
                    mlflow.pyfunc.log_model(
                        artifact_path="model",
                        python_model=DartsWrapper(),
                        artifacts=artifacts,
                        pip_requirements=["darts", "pandas", "numpy", "torch", "pytorch_lightning"],
                        input_example=pd.DataFrame({"n": [self.config.FORECAST_HORIZON]}), 
                        signature=signature,
                        # Adiciona o registro no catálogo
                        registered_model_name=full_model_name
                    )

                    # --- PARTE 3: INFERÊNCIA WALK-FORWARD (Mês a Mês) ---
                    print(f"   🔮 Iniciando Inferência Walk-Forward ({len(validation_range)} folds)...")
                    
                    for month_start in validation_range:
                        context_cutoff = month_start - pd.Timedelta(days=1)
                        metrica_mes = month_start.strftime("%Y-%m")
                        
                        val_context_series = [s.drop_after(context_cutoff) for s in full_series_scaled]
                        
                        predict_kwargs = {"n": self.config.FORECAST_HORIZON}
                        predict_kwargs['series'] = val_context_series
                        
                        if model.supports_past_covariates:
                             val_context_covs = [c.drop_after(context_cutoff) for c in full_covariates_scaled]
                             predict_kwargs['past_covariates'] = val_context_covs
                        if model.supports_future_covariates:
                             predict_kwargs['future_covariates'] = full_covariates_scaled 
                        
                        preds_scaled = model.predict(**predict_kwargs)
                        preds_inverse = target_pipeline.inverse_transform(preds_scaled, partial=True)
                        
                        metrics_month = self._calc_metrics_and_format(preds_inverse, val_series_original, metrica_mes, model_name)
                        
                        smape_m = metrics_month['metrics']['SMAPE']
                        rmse_m = metrics_month['metrics']['RMSE']
                        
                        # Log métricas mensais
                        mlflow.log_metric(f"SMAPE_{metrica_mes}", smape_m)
                        
                        print(f"     📅 {metrica_mes}: SMAPE={smape_m:.2f}%, RMSE={rmse_m:.2f}")
                        
                        all_predictions.extend(metrics_month['dfs'])

                    # --- PARTE 4: CONSOLIDAÇÃO ---
                    if all_predictions:
                        final_df = pd.concat(all_predictions)
                        final_df['VERSAO'] = self.config.VERSION
                        
                        global_mape = np.mean(np.abs((final_df['REAL'] - final_df['PREVISAO']) / final_df['REAL'])) * 100
                        global_rmse = np.sqrt(np.mean((final_df['REAL'] - final_df['PREVISAO'])**2))
                        
                        mlflow.log_metric("Global_MAPE", global_mape)
                        mlflow.log_metric("Global_RMSE", global_rmse)
                        
                        print(f"   📊 GLOBAL: MAPE={global_mape:.2f}%, RMSE={global_rmse:.2f}")
                        
                        self._save_to_delta(final_df)
                        self.success_models.append(model_name)
            
            except Exception as e:
                print(f"❌ Error training {model_name}: {e}")
                traceback.print_exc()
                self.failed_models.append(model_name)

    def _calc_metrics_and_format(self, preds, reals_full, metrica_mes, model_name):
        if not isinstance(preds, list): preds = [preds]
        if not isinstance(reals_full, list): reals_full = [reals_full]

        valid_preds, valid_reals, res_dfs = [], [], []
        
        for ts_pred, ts_real_full in zip(preds, reals_full):
            try:
                ts_real_sliced = ts_real_full.slice_intersect(ts_pred)
                valid_preds.append(ts_pred)
                valid_reals.append(ts_real_sliced)
                
                res_dfs.append(pd.DataFrame({
                    'DATA': ts_pred.time_index,
                    'PREVISAO': ts_pred.values().flatten(),
                    'REAL': ts_real_sliced.values().flatten(),
                    'CODIGO_LOJA': str(ts_pred.static_covariates.index[0]) if ts_pred.static_covariates is not None else "UNKNOWN",
                    'MODELO': model_name,
                    'METRICA_MES': metrica_mes
                }))
            except:
                continue 
        
        metrics = {"SMAPE": 0.0, "RMSE": 0.0}
        if valid_preds:
            metrics["SMAPE"] = float(np.mean(smape(valid_reals, valid_preds)))
            metrics["RMSE"] = float(np.mean(rmse(valid_reals, valid_preds)))
            
        return {"metrics": metrics, "dfs": res_dfs}

    def _save_to_delta(self, pdf):
        table_name = f"{self.config.CATALOG}.{self.config.SCHEMA}.bip_vresultado_metricas_treinamento_lojas"
        try:
            (self.config.spark_session.createDataFrame(pdf) 
             if hasattr(self.config, 'spark_session') else spark.createDataFrame(pdf)
                .write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable(table_name))
        except Exception as e:
            print(f"Save error: {e}")

/databricks/python/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


In [0]:
if spark is None:
    raise RuntimeError("Spark Session not available.")

config = Config()
config.spark_session = spark

In [0]:
# --- EXECUÇÃO DO PIPELINE (OTIMIZADO COM FEATURE STORE) ---
print(f"🚀 Iniciando Pipeline v{config.VERSION} (Walk-Forward Strict Mode)")

ingestion = DataIngestion(spark, config)

# No bloco de execução:
# 1. Busca Unificada (Feature Store + Spark ETL)
df_spark_wide = ingestion.create_training_set() # Retorna Spark DF
df_support_global = ingestion.get_global_support() # Retorna Pandas (pois é pequeno)

# 2. Construção dos Objetos Darts (Aqui ocorre o toPandas)
raw_series, raw_covs = ingestion.build_darts_objects(df_spark_wide, df_support_global)

# --- Daqui para baixo, o código original de treino se mantém igual ---
# 3. SPLIT DE TREINO
train_cutoff_date = pd.Timestamp(config.TRAIN_END_DATE) - pd.Timedelta(days=1)
print(f"✂️ Data corte para treino estático: {train_cutoff_date.date()}")

print("🛠️ Ajustando Pipeline (Scalers)...")
project_pipeline = ProjectPipeline()

train_for_fit = [s.drop_after(train_cutoff_date) for s in raw_series]
cov_for_fit = [s.drop_after(train_cutoff_date) for s in raw_covs]
project_pipeline.fit(train_for_fit, cov_for_fit)

print("🔄 Transformando TODAS as séries...")
series_scaled_full, cov_scaled_full = project_pipeline.transform(raw_series, raw_covs)

# Salvar Pipeline
pipeline_path = f"{config.PATH_SCALERS}/project_pipeline_v{config.VERSION}.pkl"
with open(pipeline_path, 'wb') as f:
    pickle.dump(project_pipeline, f)
print(f"💾 Pipeline salvo: {pipeline_path}")

# Filtragem de Séries Curtas e Sets Finais
print("🔍 Filtrando séries curtas...")
min_len = config.LAGS + config.FORECAST_HORIZON + 1
valid_indices = [i for i, ts in enumerate(train_for_fit) if len(ts) >= min_len]

train_series_static = [series_scaled_full[i].drop_after(train_cutoff_date) for i in valid_indices]
train_cov_static = [cov_scaled_full[i].drop_after(train_cutoff_date) for i in valid_indices]
full_series_valid = [series_scaled_full[i] for i in valid_indices]
full_cov_valid = [cov_scaled_full[i] for i in valid_indices]

print("🔄 Preparando targets originais para validação...")
val_series_original = project_pipeline.inverse_transform(full_series_valid, partial=True)

# 4. CONFIGURAÇÃO DE MODELOS
lag = config.LAGS
lag_covariantes = config.LAGS_FUTURE
forecast = config.FORECAST_HORIZON
lag_2 = lag + config.FORECAST_HORIZON # Lag estendido para Deep Learning
dynamic_kernel = 3 # Kernel safe size
EARLY_STOPPER = EarlyStopping(monitor="train_loss", patience=5, min_delta=0.001, mode='min')

models_dict = {
    # --- MODELOS ESTATÍSTICOS / ML CLÁSSICO ---
    "LinearRegression": LinearRegressionModel(
        lags=lag,
        lags_future_covariates=lag_covariantes,
        output_chunk_length=forecast,
        multi_models=True
    ),
    "RandomForest": RandomForest(
        lags=lag,
        lags_future_covariates=lag_covariantes,
        output_chunk_length=forecast,
        multi_models=False, # RF sklearn limitação
        random_state=42
    ),
    "LightGBM": LightGBMModel(
        lags=lag,
        lags_future_covariates=lag_covariantes,
        output_chunk_length=forecast,
        multi_models=True,
        random_state=42
    ),
    "XGBoost": XGBModel(
        lags=lag,
        lags_future_covariates=lag_covariantes,
        output_chunk_length=forecast,
        multi_models=True,
        random_state=42
    ),
    "CatBoost": CatBoostModel(
        lags=lag,
        lags_future_covariates=lag_covariantes,
        output_chunk_length=forecast,
        multi_models=True,
        random_state=42
    )
}

# --- MODELOS DE DEEP LEARNING (Adicionados se N_EPOCHS > 0) ---
if config.N_EPOCHS > 0:
    pl_trainer_kwargs = {"accelerator": "cpu", "callbacks": [EARLY_STOPPER]}
    models_dict.update({
        "TFT": TFTModel(
            input_chunk_length=lag_2,
            output_chunk_length=forecast,
            hidden_size=128,
            lstm_layers=2,
            num_attention_heads=4,
            dropout=0.2,
            batch_size=4,
            n_epochs=config.N_EPOCHS,
            add_relative_index=True,
            random_state=42,
            pl_trainer_kwargs=pl_trainer_kwargs
        ),
        "NBEATS": NBEATSModel(
            input_chunk_length=lag_2,
            output_chunk_length=forecast,
            generic_architecture=True,
            num_stacks=3,
            num_blocks=3,
            num_layers=4,
            layer_widths=256,
            batch_size=4,
            n_epochs=config.N_EPOCHS,
            random_state=42,
            pl_trainer_kwargs=pl_trainer_kwargs
        ),
        "Transformer": TransformerModel(
            input_chunk_length=lag_2,
            output_chunk_length=forecast,
            d_model=128,
            nhead=4,
            num_encoder_layers=3,
            num_decoder_layers=3,
            dim_feedforward=256,
            dropout=0.2,
            batch_size=4,
            n_epochs=config.N_EPOCHS,
            random_state=42,
            pl_trainer_kwargs=pl_trainer_kwargs
        ),
        "BlockRNN": BlockRNNModel(
            model='LSTM',
            input_chunk_length=lag_2,
            output_chunk_length=forecast,
            hidden_dim=128,
            n_rnn_layers=2,
            dropout=0.2,
            batch_size=4,
            n_epochs=config.N_EPOCHS,
            random_state=42,
            pl_trainer_kwargs=pl_trainer_kwargs
        ),
        "TCN": TCNModel(
            input_chunk_length=lag_2,
            output_chunk_length=forecast,
            kernel_size=dynamic_kernel,
            num_filters=lag_2,
            num_layers=None,
            dilation_base=2,
            dropout=0.2,
            batch_size=4,
            n_epochs=config.N_EPOCHS,
            random_state=42,
            pl_trainer_kwargs=pl_trainer_kwargs
        )
    })

trainer = ModelTrainer(config, models_dict)
trainer.train_evaluate_walkforward(
    train_series_static=train_series_static,
    train_covs_static=train_cov_static,
    full_series_scaled=full_series_valid,
    full_covariates_scaled=full_cov_valid,
    val_series_original=val_series_original,
    target_pipeline=project_pipeline
)
print("✅ Processo Finalizado.")

🚀 Iniciando Pipeline v2026_01_15_09_42 (Walk-Forward Strict Mode)
🛒 Construindo Training Set via Feature Store (Spark Native)...


2026/01/15 12:42:41 WARNING databricks.ml_features.entities.feature_lookup: The output_name parameter is deprecated.  Use "rename_outputs".


   ⚡ Executando limpeza e tratamento no Spark Cluster...
🌍 Carregando suporte global (Spark Aggregation)...
⚙️ Materializando dados do Spark para Pandas (Driver)...
   Build: Criando Target Series...
   Build: Criando Covariáveis Locais...
   Build: Stacking Final...
✅ Objetos Darts Prontos: 1 lojas.
✂️ Data corte para treino estático: 2024-12-31
🛠️ Ajustando Pipeline (Scalers)...
🔄 Transformando TODAS as séries...
💾 Pipeline salvo: /Volumes/ds_dev/cvc/experiments/artefacts/loja/scalar/validation/project_pipeline_v2026_01_15_09_42.pkl
🔍 Filtrando séries curtas...
🔄 Preparando targets originais para validação...

🚀 [Model: LinearRegression] Iniciando Processo...
   📝 Registrando metadados do experimento...
   🏋️ Treinando com dados até 2025-01-01...


2026/01/15 12:45:51 INFO mlflow.pyfunc: Validating input example against model signature


   💾 Registrando modelo como: ds_dev.cvc.loja_LinearRegression


...

Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


     📅 2025-04: SMAPE=200.00%, RMSE=12944.18


Predicting: |          | 0/? [00:00<?, ?it/s]

     📅 2025-05: SMAPE=166.75%, RMSE=7710.85
   📊 GLOBAL: MAPE=inf%, RMSE=8172.27
✅ Processo Finalizado.
